In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

import numpy as np
import matplotlib.pyplot as plt

import pyarrow.parquet as pq
import pyarrow as pa

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, IncrementalPCA, SparsePCA
import pandas as pd
np.set_printoptions(suppress=True)

# Eugen stuff
import sklearn.svm
import sklearn

Using TensorFlow backend.


In [3]:
samples = pq.read_table('/data/gtex/samples.parquet').to_pandas()
genes = pd.read_csv("/data/gtex/genage_gtex.tsv", sep="\t", index_col=0, header=0)

In [4]:
data = genes.T
data.columns.names = ["Name"]
data.head(5)

Name,ENSG00000198804,ENSG00000196689,ENSG00000170899,ENSG00000259384,ENSG00000096968,ENSG00000112964,ENSG00000100311,ENSG00000127445,ENSG00000141837,ENSG00000118432,...,ENSG00000073756,ENSG00000143252,ENSG00000115966,ENSG00000169047,ENSG00000183395,ENSG00000017427,ENSG00000177606,ENSG00000164104,ENSG00000160957,ENSG00000059728
GTEX-1117F-0226-SM-5GZZ7,10660.0,20.67,24.83,0.2450,23.78,37.70,23.76,37.33,0.6694,6.685,...,2.1360,43.67,23.13,11.47,0.1898,40.63,526.4,139.50,3.667,16.590
GTEX-111CU-1826-SM-5GZYN,11060.0,10.58,17.71,0.4593,21.51,73.55,34.54,42.96,1.0900,29.800,...,0.5251,71.07,22.71,10.37,0.4575,68.23,256.6,95.25,3.141,12.520
GTEX-111FC-0226-SM-5N9B8,21320.0,15.66,30.25,4.6720,22.64,23.37,70.91,38.26,1.9000,10.750,...,1.1170,56.96,32.70,13.02,0.0000,24.38,519.3,76.09,3.941,8.683
GTEX-111VG-2326-SM-5N9BK,19850.0,12.31,16.50,4.8020,19.02,15.95,41.83,33.27,0.7103,1.704,...,6.1840,52.94,21.67,5.35,0.1227,29.26,871.3,69.30,5.417,37.350
GTEX-111YS-2426-SM-5GZZQ,31260.0,6.76,16.79,5.4420,31.18,116.20,71.35,27.91,1.0790,26.620,...,0.3932,44.61,23.08,10.55,0.8340,33.65,175.3,98.29,2.780,9.886


In [5]:
samples = samples.set_index("Name")
j = samples.join(data)
j.head(5)

,Subject,Tissue,Subtissue,Sex,Age,Death,Avg_age,ENSG00000198804,ENSG00000196689,ENSG00000170899,...,ENSG00000073756,ENSG00000143252,ENSG00000115966,ENSG00000169047,ENSG00000183395,ENSG00000017427,ENSG00000177606,ENSG00000164104,ENSG00000160957,ENSG00000059728
Name,,,,,,,,,,,,,,,,,,,,,
GTEX-11O72-2926-SM-5BC4V,GTEX-11O72,Brain,Brain - Cortex,1,40-49,2.0,44.5,80890.0,5.281,34.110,...,4.3410,20.27,19.450,2.518,0.0000,0.47010,93.79,21.86,6.599,8.137
GTEX-11TTK-0008-SM-5S2RU,GTEX-11TTK,Skin,Cells - Transformed fibroblasts,2,60-69,4.0,64.5,23710.0,4.964,43.160,...,0.8661,121.50,26.410,40.600,0.1677,0.08062,81.60,143.90,6.836,2.439
GTEX-11UD2-0226-SM-5EQKY,GTEX-11UD2,Skin,Skin - Sun Exposed (Lower leg),1,50-59,2.0,54.5,31680.0,5.967,65.190,...,4.8050,54.38,9.462,8.703,0.1313,1.18900,302.20,53.00,5.655,22.060
GTEX-12KS4-0005-SM-5SI94,GTEX-12KS4,Blood,Whole Blood,1,30-39,0.0,34.5,3867.0,1.801,1.044,...,10.1800,35.87,9.118,0.215,0.0000,0.32690,100.40,165.60,6.525,737.900
GTEX-131XE-2526-SM-5EQ57,GTEX-131XE,Nerve,Nerve - Tibial,1,50-59,0.0,54.5,12650.0,12.470,32.010,...,1.3920,69.37,25.880,10.210,0.3869,11.71000,535.00,97.57,12.400,16.790


In [6]:
ji = j.columns.drop(["Subject", "Tissue", "Subtissue", "Sex", "Age", "Death", "Avg_age"])

In [7]:
Y = j["Avg_age"].to_numpy()

In [45]:
X = j[ji].to_numpy()

In [48]:
x = keras.utils.normalize(X,axis=0)


NameError: name 'keras' is not defined

In [44]:
np.isnan(Y).sum()

0

In [35]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_x.fit(X)
scaler_y.fit(Y)
xscale=scaler_x.transform(X)
yscale=scaler_y.transform(Y)


ValueError: Expected 2D array, got 1D array instead:
array=[44.5 64.5 54.5 ... 64.5 54.5 64.5].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [16]:
X.shape[1]

297

In [14]:
(trainX, testX, trainY, testY) = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=297, activation='relu'))
model.add(Dense(12, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation='linear'))
model.summary()

In [ ]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 40

# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

In [32]:
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=32)

Train on 9350 samples, validate on 2338 samples
Epoch 1/40
9350/9350 [==============================] - 1s 85us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/40
9350/9350 [==============================] - 1s 54us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/40
9350/9350 [==============================] - 1s 58us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/40
9350/9350 [==============================] - 1s 57us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/40
9350/9350 [==============================] - 1s 55us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 6/40
9350/9350 [==============================] - 1s 61us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/40
9350/9350 [==============================] - 1s 60us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.00